In [36]:
import os
from dotenv import load_dotenv
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import time
from openai import OpenAI
from prompts import question_prompt, assessment_prompt
# Load environment variables
load_dotenv()


# Access email and password from .env file
email_id = os.getenv("EMAIL")
password = os.getenv("PASSWORD")

def login():
    # Wait until the email input field is present
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, '/html/body/div[1]/section/div/div[2]/div/div/div/form/fieldset/div[1]/input'))
    )
    # Find the email input field and enter the email
    email_input = driver.find_element(By.XPATH, '/html/body/div[1]/section/div/div[2]/div/div/div/form/fieldset/div[1]/input')
    email_input.send_keys(email_id)
    # Submit the email
    email_input.send_keys(Keys.RETURN)

    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, '/html/body/div[1]/section/div/div[2]/div/div/div/form/fieldset/div[2]/input'))
    )
    # Find the password input field and enter the password
    password_input = driver.find_element(By.XPATH, '/html/body/div[1]/section/div/div[2]/div/div/div/form/fieldset/div[2]/input')
    password_input.send_keys(password)
    # Submit the form
    password_input.send_keys(Keys.RETURN)

    

def click_optional_buttons():
    # optional button 1
    # Resume Course section button.
    try:
        button = driver.find_element(By.XPATH, '/html/body/div[3]/section/div[1]/div[1]/div/a')
        button.click()
    except:
        pass
    try:
        button = driver.find_element(By.XPATH, '/html/body/div[3]/section/div/div/div/div/div/a[1]')
        button.click()
    except:
        pass
    
    
    
def get_question_section(xpath="/html/body/div[3]/section/div/div"):
    # Check if the specific section exists
    try:
        section = driver.find_element(By.XPATH, xpath)
#         if section.get_attribute("data-test") == "popupQuestionContainer":
#             print("The Question section exists on the page.")
        question_section_html = section.get_attribute('outerHTML')
        return question_section_html
    except:
        pass
    return None

def get_assessment_section(xpath='/html/body/div[3]/section/div/div'):
    try:
        section = driver.find_element(By.XPATH, xpath)
#         if section.get_attribute("class") == "gritCourseflowNode__question-wrapper":
#             print(True)
#             print("The Question section exists on the page.")
        assessment_section_html = section.get_attribute('outerHTML')
        return assessment_section_html
    except:
        pass
    return None


# def answer_question(answer_button_Xpath):
#     try:
#         answer_button = WebDriverWait(driver, 10).until(
#             EC.element_to_be_clickable((By.XPATH, answer_button_Xpath))
#         )
#         answer_button.click()
#         print("Clicked on the answer button.")
#         try:
#             submit_button = WebDriverWait(driver, 10).until(
#                 EC.element_to_be_clickable((By.XPATH, '/html/body/div/section/section/main/main/nav/div/div[3]/button'))
#             )
#             submit_button.click()
#             print("Clicked on the submit button.")
#             answer_status = answer_status = WebDriverWait(driver, 10).until(
#                     EC.element_to_be_clickable((By.XPATH,'/html/body/div/section/section/main/main/article/div/div[2]/div/div[1]/div[2]/div/div/h3'))
#                 ).text
#             return answer_status
#         except Exception as e:
#             print(f"An error occurred while clicking the submit button: {e}")
#     except Exception as e:
#         print(f"An error occurred while clicking the answer button: {e}")
#     return None

# def answer_assessment(answer_text):
#     try:
#         escaped_text = answer_text.replace("'", "\'")
    
#         # Use an XPath with normalize-space() and contains() to handle multiline and special characters
#         button = driver.find_element(By.XPATH, f"//button[contains(normalize-space(), \"{escaped_text}\")]")
#         button.click()
#         print(f"Clicked the button with text {answer_text}.")
#         try:
#             submit_button = WebDriverWait(driver, 10).until(
#                 EC.element_to_be_clickable((By.XPATH, '/html/body/div/section/section/main/div/nav/div/div[3]/button'))
#             )
#             submit_button.click()
#             print("Clicked on the submit button.")
#         except Exception as e:
#             print(f"Error occured while clicking Submit. {e}")
#     except Exception as e:
#         print(f"An error occurred: {e}")


In [3]:
from openai import OpenAI

def get_gpt4o_response(prompt, model="gpt-4o", temperature=0, max_tokens=300):
    """
    Function to send a prompt to the OpenAI GPT-4 model and retrieve a response.

    Parameters:
        prompt (str): The input prompt to send to the GPT model.
        model (str): The model to use (default is "gpt-4").
        temperature (float): Sampling temperature. Higher values make the output more random.
        max_tokens (int): The maximum number of tokens to include in the response.

    Returns:
        str: The model's response to the input prompt.
    """
    try:
        
        client = OpenAI(
            api_key=os.getenv("OPENAI_API_KEY")
        )
        # Send the prompt to the OpenAI API
        response = client.chat.completions.create(
            messages=[{"role": "user", "content": prompt,}],
            temperature=temperature,
            model=model,
        )

        # Extract and return the response content
        return response.choices[0].message.content
    except Exception as e:
        return f"An error occurred: {str(e)}"


In [4]:
# Initialize WebDriver (use ChromeDriver as an example)
service = Service()  # Update with the path to your ChromeDriver
options = webdriver.ChromeOptions()

# Optional: Add any additional options (e.g., headless mode)
# options.add_argument('--headless')

# Start the WebDriver
driver = webdriver.Chrome(service=service, options=options)
driver.get("https://fusion.certus.com/courses")
# login to the account
login()
# Wait for the page to load after login
page = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.TAG_NAME, 'body'))
)


In [51]:
# Click Next Button
try:
    driver.switch_to.default_content()
    driver.switch_to.default_content()

    timer_text = driver.find_element(By.XPATH, '/html/body/div[3]/section/div[2]/div/div[2]/div/span[3]').text
    print(timer_text)
    if timer_text !='':
        h,m,s = [int(x) for x in timer_text.split(":")]
        wait_time = (h*3600)+(m*60)+s
        print(f"Waiting for {wait_time+1} seconds")
        time.sleep(wait_time+1)

    next_button = driver.find_element(By.XPATH, '/html/body/div[3]/section/div[2]/div/div[5]/div/button')
    print("Clicking next button...")
    WebDriverWait(driver, 10).until(EC.element_to_be_clickable(next_button)).click()
except Exception as e:
    print(e)

00:09:49
Waiting for 590 seconds
Clicking next button...


In [47]:
# Click Show More Button
try:
    driver.switch_to.default_content()
    driver.switch_to.default_content()

    iframe1 = driver.find_element(By.XPATH, "/html/body/div[3]/section/div[2]/div/div[4]/div/iframe")
    driver.switch_to.frame(iframe1)
    iframe2 = driver.find_element(By.XPATH, "/html/body/iframe[1]")
    driver.switch_to.frame(iframe2)
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(2)
    
    show_more_button = driver.find_element(By.XPATH, "/html/body/div[1]/div/div[11]/div[2]/div[2]/div[4]/div/div/div/div[3]/a")
#     show_more_button = driver.find_element(By.XPATH, '/html/body/div[2]/div/div[11]/div[2]/div[2]/div[4]/div/div/div/div[3]/a')
    WebDriverWait(driver, 10).until(EC.element_to_be_clickable(show_more_button)).click()

    driver.switch_to.default_content()
    driver.switch_to.default_content()
except Exception as e:
    print("Button could not be clicked")
    print(e)

Message: 



In [ ]:
# Click options in question
# option = driver.find_element(By.XPATH, "/html/body/div[3]/section/div/div/div/div[2]/form/div[1]/div/div[5]/label/text()")
# option.click()

In [ ]:
<div class="questions">
    <h4 class="question" data-ext-id="hdr-securityquestions-q117">What was the first instrument you learned to play?</h4>
        <div class="choices" data-ext-id="div-securityquestions-q117-choices">
            <input type="hidden" value="117" name="QuestionId">
                <div class="custom-control custom-radio">
                    <input type="radio" required="" value="510" id="answer-510" name="AnswerId" class="custom-control-input" data-ext-id="rdo-securityquestions-q117-c510">
                    <label class="custom-control-label" for="answer-510" data-ext-id="lbl-securityquestions-q117-c510">
                        Piano
                    </label>
                </div>
                <div class="custom-control custom-radio">
                    <input type="radio" required="" value="511" id="answer-511" name="AnswerId" class="custom-control-input" data-ext-id="rdo-securityquestions-q117-c511">
                    <label class="custom-control-label" for="answer-511" data-ext-id="lbl-securityquestions-q117-c511">
                        Violin
                    </label>
                </div>
                <div class="custom-control custom-radio">
                    <input type="radio" required="" value="512" id="answer-512" name="AnswerId" class="custom-control-input" data-ext-id="rdo-securityquestions-q117-c512">
                    <label class="custom-control-label" for="answer-512" data-ext-id="lbl-securityquestions-q117-c512">
                        Guitar
                    </label>
                </div>
                <div class="custom-control custom-radio">
                    <input type="radio" required="" value="513" id="answer-513" name="AnswerId" class="custom-control-input" data-ext-id="rdo-securityquestions-q117-c513">
                    <label class="custom-control-label" for="answer-513" data-ext-id="lbl-securityquestions-q117-c513">
                        Drums
                    </label>
                </div>
                <div class="custom-control custom-radio">
                    <input type="radio" required="" value="514" id="answer-514" name="AnswerId" class="custom-control-input" data-ext-id="rdo-securityquestions-q117-c514">
                    <label class="custom-control-label" for="answer-514" data-ext-id="lbl-securityquestions-q117-c514">
                        I never learned an instrument
                    </label>
                </div>
        </div>
</div>

/html/body/div[3]/section/div/div/div/div[2]/form/div[1]

In [ ]:
I never learned an instrument

/html/body/div[3]/section/div/div/div/div[2]/form/div[1]/div/div[5]/label

In [ ]:
/html/body/div[3]/section/div/div/div/div/div/a[1]

In [59]:
click_optional_buttons()

In [ ]:
# def check_question_section(xpath):
#     # Check if the specific section exists
#     try:
#         section = driver.find_element(By.XPATH, xpath)
# #         if section.get_attribute("data-test") == "popupQuestionContainer":
# #             print("The Question section exists on the page.")
#         question_section_html = section.get_attribute('outerHTML')
#         return question_section_html
#     except:
#         pass
#     return None


# xpath = '/html/body/div[3]/section/div/div[2]/div/form/div[1]'
# check_question_section(xpath)

In [ ]:
xpath = '/html/body/div[3]/section/div/div[2]/div/form/div[1]'

question_section_html = check_question_section(xpath)
prompt = question_prompt.format(xpath = xpath, question_section_html=question_section_html)

print(f"Pinging gpt4o...\n\n{prompt}\n\n")
answer_button_Xpath = get_gpt4o_response(prompt).strip()

print(f"Response:{answer_button_Xpath}\n\n")
# answer_status = answer_question(answer_button_Xpath)

In [ ]:
prompt = assessment_prompt.format(xpath=xpath, assessment_section_html=question_section_html)
print(f"Pinging gpt4o...\n\n{prompt}\n\n")

response = get_gpt4o_response(prompt).strip()
if "python" in response:
    answer_text = response
else:
    answer_text = response.strip()
print(f"Response: {answer_text}\n\n")
# answer_status = answer_assessment(answer_text)


In [ ]:
click_optional_buttons()

In [ ]:
show_more_button = driver.find_element(By.XPATH, "/html/body/div[3]/section/div[2]/div/div[4]/div/iframe")

In [ ]:
show_more_button = driver.find_element(By.XPATH, "/html/body/div[2]/div/div[11]/div[2]/div[2]/div[4]/div/div/div/div[3]/a")

In [ ]:
WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.XPATH, '/html/body/div[3]/section/div[2]/div/div[5]/div/button'))
)
next_button = driver.find_element(By.XPATH, '/html/body/div[3]/section/div[2]/div/div[5]/div/button')
next_button.click()


In [ ]:
while True:
    click_optional_buttons()

#     # Answer Questions during Course
#     question_section_html = check_question_section()
#     if question_section_html!=None:
#         prompt = question_prompt.format(question_section_html=question_section_html)
#         print(f"Pinging gpt4o...\n\n{prompt}\n\n")
#         answer_button_Xpath = get_gpt4o_response(prompt).strip().split("'")[-2]
#         print(f"Response:{answer_button_Xpath}\n\n")
#         answer_status = answer_question(answer_button_Xpath)
# #         if "Correct" in answer_status:
# #             print("Answer is Correct")
# #         else:
# #             print("Answer is Wrong")
# #             break

#     # Answer Assessment Questions
#     assessment_section_html = check_assessment_section()
#     if assessment_section_html!=None:
#         prompt = assessment_prompt.format(assessment_section_html=assessment_section_html)
#         print(f"Pinging gpt4o...\n\n{prompt}\n\n")
#         response = get_gpt4o_response(prompt).strip()
#         if "python" in response:
#             answer_text = response.split('"')[-2]
#         else:
#             answer_text = response.strip().split("'")[-2]
#         print(f"Response: {answer_text}\n\n")
#         answer_status = answer_assessment(answer_text)

    # Wait for the timer countdown to finish and click the button
    try:
        next_button = driver.find_element(By.XPATH, '/html/body/div/section/section/main/main/nav/div/div[3]/button')
        try:
            timer_element = driver.find_element(By.XPATH, '/html/body/div/section/section/main/main/nav/div/div[3]/button/span[1]/div')
            timer_text = timer_element.text
            
            # Parse the timer text into seconds
            if ":" in timer_text:
                minutes, seconds = map(int, timer_text.split(":"))
                wait_time = minutes * 60 + seconds
            else:
                wait_time = int(timer_text)

            # Wait for the specified time
            print(f"Waiting for {wait_time} seconds for the timer to reach 00:00.")
            time.sleep(wait_time)
            next_button.click()
        except:
            print("Timer not found, just clicking next button")
            # Click the button

            next_button.click()

    except Exception as e:
        print(f"An error occurred while waiting for the timer or clicking the next button: {e}")
    time.sleep(3)
